In [1]:
import sqlalchemy as sql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import scipy as sci

### Test with general optimization procedures with the CVXPY and CVXOPT

In [2]:

import cvxpy as cp
from cvxopt import matrix, solvers
import bt

### Set General Paths for loading modules and variables

In [3]:
home_dir = os.path.expandvars("$HOME")
app_src_dir = '/dev/repos/FinancialAnalyticsSystem/src'
sys.path.insert(0, home_dir + app_src_dir)

In [4]:
%load_ext autoreload
%autoreload 1

In [5]:
#from DataHandlingSystem.DataLoadingSystem import *
%aimport DataHandlingSystem
%aimport FeatureExtractionSystem

In [6]:
from DataHandlingSystem.DataLoadingSystem import *
from FeatureExtractionSystem.DeltaFeatureExtractor import * 

In [7]:
sql_pass = str(np.loadtxt('/home/l7/dev/docs/sql_login.txt', dtype=str))
db_name = 'stocks_eod'
owner = 'sql_manager'
data_loader = DataLoadingSystem(db_owner=owner, db_name=db_name, sql_pass=sql_pass)


/home/l7/dev/tools/anaconda3/envs/analytics_system_env_3.6.5/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [337]:
start_date = "2004-12-28"
end_date = '2019-01-01'
table_names = ['NVDA', # 'MSFT', 'AXP', 'BA', 'CAT', 'ASML',
              'DAI_DE', 'MC_PA', 'UPM_HE', 'FORTUM_HE']
data_container = {}
smallest_number_of_entries = np.inf
smallest_table = ''
for table_name in table_names:
    #print(table_name)
    data = data_loader.get_data_for_specific_date_range(table_name=table_name,
                                    start_date=start_date,
                                    end_date=end_date)
    data_container[table_name] = data
    #print(data_container[table_name].index.size)
    if data_container[table_name].index.size < smallest_number_of_entries:
        smallest_number_of_entries = data_container[table_name].index.size
        smallest_table = table_name
#data_container

In [338]:
change_df = pd.DataFrame(columns=data_container.keys())
distance = 1
lsuffix = '_x'
change_df[smallest_table] = np.append(np.zeros([1,distance]),get_change_date_range(data=data_container[smallest_table]['adjusted close'].values,
                                                             distance=distance))
for table_name in table_names:
    if table_name == smallest_table:
        pass
    else:
        merged_data = data_container[table_name].join(data_container[smallest_table],
                                             how='right', 
                                             lsuffix=lsuffix)
        change_df[table_name] = np.append(np.zeros([1,distance]),get_change_date_range(data=merged_data['adjusted close' + lsuffix].values,
                                                                 distance=distance))
change_df = change_df.replace([np.inf, -np.inf], np.nan)
change_df = change_df.dropna(how='any')
#change_df

Daily $\mathbb{E}(r)$, as well as $\Sigma$

In [339]:
# TODO: These tests account strickly for daily changes. If we would llike to 
# look at changes over some period of time, it is necessary to recompute the 
# the data frame containing changes for one which looks at changes over
# longer period. 
expected_returns = change_df[distance:].mean()
print("Number of observations: {}".format(change_df.index.size))
print("expected returns: {}".format(expected_returns))
sigma = np.cov(change_df[distance:].values, rowvar=False)
print("sigma: {}".format(sigma))
N = sigma.shape[0]
print("N: {}".format(N))
print('sum of sigma: {}'.format(np.sum(sigma)))

Number of observations: 3346
expected returns: NVDA         0.001345
DAI_DE       0.000520
MC_PA        0.000668
UPM_HE       0.000543
FORTUM_HE    0.000572
dtype: float64
sigma: [[8.87080436e-04 1.71986142e-04 1.43354774e-04 1.27629380e-04
  9.46416956e-05]
 [1.71986142e-04 4.39279112e-04 2.39222477e-04 2.46247239e-04
  1.47920742e-04]
 [1.43354774e-04 2.39222477e-04 3.11643822e-04 1.94919073e-04
  1.27436192e-04]
 [1.27629380e-04 2.46247239e-04 1.94919073e-04 4.29136614e-04
  1.46490424e-04]
 [9.46416956e-05 1.47920742e-04 1.27436192e-04 1.46490424e-04
  3.07515092e-04]]
N: 5
sum of sigma: 0.005654351351106666


In [340]:
correlations = np.corrcoef(change_df[distance:].values, rowvar=False)
print(correlations)
min_row = np.argmin(correlations)
#np.min(correlations)b

[[1.         0.27551273 0.27264742 0.20685755 0.18120401]
 [0.27551273 1.         0.64655049 0.56715702 0.40246265]
 [0.27264742 0.64655049 1.         0.5330001  0.41165207]
 [0.20685755 0.56715702 0.5330001  1.         0.40325358]
 [0.18120401 0.40246265 0.41165207 0.40325358 1.        ]]


In [341]:
t = np.array([[1,2],[1, 1]])
a = np.array([1,2])
a @ t

array([3, 4])

In [342]:

r = expected_returns.values.reshape([N,1])
b = 1
ones = lambda y: np.ones([y, 1])

In [350]:
def calculate_mdp(sigma):
    """
    Maximum Diversified Portfolio
    :param sigma: covariance matrix
    :param min_gmvp_point: paramter that stipulates what is the minimum gmpv point at which
    to place as constraint in the QCP
    :return:
    """
    N = sigma.shape[0]
    # bp()
    volatilities = np.sqrt(np.diag(sigma))
    volatilities = volatilities.reshape(volatilities.shape[0], 1)
    w_N = np.ones([N, 1]) * 1 / N
    V_N = w_N.T  @ volatilities
    D_N = 100
    R_N = V_N / D_N
    x = cp.Variable(shape=(N,1))
    V = x.T @ volatilities # cp.quad_form(x, sigma)
    constraints = [x >= np.ones([N,1]) * 0,
                   np.ones([N,1]).T @ x == 1,
                   cp.quad_form(x, sigma) <= R_N,
                   ]
    problem = cp.Problem(cp.Maximize(V),
                         constraints)
    optimum = problem.solve(#solver='OSQP',
                            qcp=True)
    gmvp_point = cp.quad_form(x, sigma).value
    return x.value, optimum, gmvp_point

In [348]:
volatilities = np.sqrt(np.diag(sigma)) 
w_N = np.ones([N, 1]) * 1 / N
V_N = w_N.T  @ volatilities
#D_N = V_N / np.sqrt(w_N.T @ sigma @ w_N)
D_N = 100
R_N = V_N / D_N #np.sqrt(w_N.T @ sigma @ w_N)
print("D_N: {}, V_N: {}, R_N: {}, R_N^2: {}".format(D_N, V_N, R_N, R_N**2))

D_N: 100, V_N: [0.02132961], R_N: [0.0002133], R_N^2: [4.54952174e-08]


In [349]:
N = sigma.shape[0]
min_gmvp_point =  R_N #(np.sum(sigma) / N) ** 1.2
print('N: {}, 1/N: {}, min_gmvp_point: {}'.format(N, 1/N, min_gmvp_point))
diversificaton_ratio = 100
w, D, gmvp_point = calculate_mdp(sigma)
print("weights: {}, \n D: {}, gmvp point achieved: {}".format(w, D / gmvp_point, gmvp_point))

N: 5, 1/N: 0.2, min_gmvp_point: [0.0002133]
weights: [[0.2089542 ]
 [0.08670987]
 [0.18811979]
 [0.17996944]
 [0.3362467 ]], 
 D: [[98.39101497]], gmvp point achieved: [[0.0002133]]


In [346]:
x = cp.Variable(shape=(N,1))
print(x.shape)
#R = 0.05
R = cp.Variable(shape=(1,))
problem = cp.Problem(cp.Minimize((1/2) * cp.quad_form(x, sigma)),
                     [x >= np.ones([N,1]) * 0,
                      ones(N).T @ x == b,
                      # R >=0.10,
                     #r.T @ x >= R
                     ])

(5, 1)


In [109]:
print("variance of portfolio: {}".format(problem.solve()))
print(x.value.round(3))
print(np.sum(x.value))
print("Value of return : {}".format(R.value))

variance of portfolio: 0.00186308381805147
[[-0.   ]
 [ 0.411]
 [-0.   ]
 [ 0.151]
 [-0.   ]
 [ 0.258]
 [ 0.   ]
 [ 0.181]]
0.9999999999999999
Value of return : None


In [ ]:
x = cp.Variable(shape=(N,1))
R = cp.Variable(shape=(1,))
accepted_volatility = 0.30
minimum_expected_return = 0.15
portfolio_variance = (1/2) * cp.quad_form(x, sigma) <= accepted_volatility ** 2
problem = cp.Problem(cp.Maximize(R),
                     [x >= np.ones([N,1]) * 0,
                      ones(N).T @ x == b,
                      R >=minimum_expected_return,
                      r.T @ x >= R,
                      portfolio_variance])

In [ ]:
problem.solve()
final_variance = x.T.value @ sigma @ x.value
print("final variance: {}".format(final_variance))
print(x.value.round(3))
print(np.sum(x.value))
print("Value of return : {}".format(R.value))

In [ ]:
labels = []
for item in expected_returns.index.values:
    labels.append(item)
labels

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Expected return vs Volatility')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.scatter(accepted_volatility, minimum_expected_return,
            label='Minima accepted', s=200, marker='3')
plt.annotate('Minima accepted', (accepted_volatility, minimum_expected_return))
plt.scatter(x.value.T @ sigma @ x.value, R.value,
            label='sum achieved', s=200, marker='3')
plt.annotate('sum achieved', (x.value.T @ sigma @ x.value,
                                 R.value))
volatility = x.value * np.sqrt(x.value)
final_expected_return = x.value * R.value
for _x, _y , label in zip(volatility, final_expected_return, labels):
    plt.scatter(_x, _y, label=label, s=_y * 1000)
    #if _x > 0.01:
        #plt.annotate(label, (_x, _y))

plt.legend()
plt.show()

In [ ]:
data = bt.get('DAI.DE, UPM.HE, MC.PA', start='2010-01-01')
print(data.head())

In [ ]:
data.columns

In [ ]:
s = bt.Strategy('s1', [bt.algos.RunMonthly(),
                      bt.algos.SelectAll(),
                      bt.algos.WeighEqually(),
                      bt.algos.Rebalance()])

In [ ]:
test = bt.Backtest(s, data)
res = bt.run(test)

In [ ]:
res.plot()

In [ ]:
res.plot_histogram()

In [ ]:
res.stats

In [ ]:
res.plot_security_weights()

In [ ]:
from pandas_datareader import data as pdata

In [ ]:
fortum = pdata.get_data_yahoo('FORTUM.HE', start='2010-01-01', end='2019-01-01')

In [ ]:
fortum